In [1]:
pip install yfinance


Note: you may need to restart the kernel to use updated packages.


In [4]:
import yfinance as yf

# Alapértelmezett küszöbértékek a pénzügyi mutatókhoz.
default_thresholds = {
    "trailingPE": {"ideal": 15, "borderline": 25, "direction": "lower"},
    "forwardPE": {"ideal": 15, "borderline": 25, "direction": "lower"},
    "pegRatio": {"ideal": 1, "borderline": 2, "direction": "lower"},
    "returnOnEquity": {"ideal": 0.15, "borderline": 0.10, "direction": "higher"},
    "profitMargins": {"ideal": 0.10, "borderline": 0.05, "direction": "higher"},
    "dividendYield": {"ideal": 0.03, "borderline": 0.02, "direction": "higher"}
}

# Szektor-specifikus módosítások
sector_adjustments = {
    "Technology": {  # Informatikai szektor
         "trailingPE": {"ideal": 25, "borderline": 40},
         "forwardPE": {"ideal": 25, "borderline": 40}
    },
    "Financials": {  # Pénzügyi szektor
         "trailingPE": {"ideal": 10, "borderline": 15},
         "forwardPE": {"ideal": 10, "borderline": 15},
         "returnOnEquity": {"ideal": 0.12, "borderline": 0.08}
    },
    "Industrials": {  # Ipari szektor
         "trailingPE": {"ideal": 18, "borderline": 30},
         "forwardPE": {"ideal": 18, "borderline": 30}
    },
    "Energy": {  # Energia szektor
         "trailingPE": {"ideal": 10, "borderline": 20},
         "forwardPE": {"ideal": 10, "borderline": 20}
    },
    # További, a piacon jelentős szektorok:
    "Consumer Discretionary": {
         "trailingPE": {"ideal": 15, "borderline": 25},
         "forwardPE": {"ideal": 15, "borderline": 25}
    },
    "Consumer Staples": {
         "trailingPE": {"ideal": 15, "borderline": 25},
         "forwardPE": {"ideal": 15, "borderline": 25}
    },
    "Health Care": {
         "trailingPE": {"ideal": 20, "borderline": 30},
         "forwardPE": {"ideal": 20, "borderline": 30}
    },
    "Communication Services": {
         "trailingPE": {"ideal": 18, "borderline": 30},
         "forwardPE": {"ideal": 18, "borderline": 30}
    },
    "Materials": {
         "trailingPE": {"ideal": 15, "borderline": 25},
         "forwardPE": {"ideal": 15, "borderline": 25}
    },
    "Utilities": {
         "trailingPE": {"ideal": 20, "borderline": 30},
         "forwardPE": {"ideal": 20, "borderline": 30},
         "dividendYield": {"ideal": 0.04, "borderline": 0.03, "direction": "higher"}
    },
    "Real Estate": {
         "trailingPE": {"ideal": 20, "borderline": 30},
         "forwardPE": {"ideal": 20, "borderline": 30}
    }
}

def get_threshold(metric, sector):
    """
    Visszaadja az adott pénzügyi mutatóhoz tartozó küszöbértékeket.
    Ha a szektorhoz van speciális beállítás, azt alkalmazza.
    """
    base = default_thresholds.get(metric, {}).copy()
    if sector in sector_adjustments and metric in sector_adjustments[sector]:
        adjustment = sector_adjustments[sector][metric]
        base["ideal"] = adjustment["ideal"]
        base["borderline"] = adjustment["borderline"]
        if "direction" in adjustment:
            base["direction"] = adjustment["direction"]
    return base

def evaluate_fundamentals(info):
    """
    Az alábbi fundamentális mutatók alapján pontozza a vállalatot:
      - Trailing P/E
      - Forward P/E
      - PEG Ratio
      - Return on Equity (ROE)
      - Profit Margin
      - Dividend Yield
      - Revenue Growth
      - Debt-to-Equity
  
    Minden mutatónál a megfelelő küszöbértékek alapján járul hozzá a végső pontszámhoz.
    """
    score = 0
    sector = info.get("sector", "Unknown")
    print("Szektor:", sector)
    print("\n--- Fundamentális mutatók ---")
    
    # Trailing P/E
    pe = info.get("trailingPE")
    if pe is not None:
        print("Trailing P/E:", pe)
        thresholds = get_threshold("trailingPE", sector)
        if thresholds["direction"] == "lower":
            if pe < thresholds["ideal"]:
                score += 2
            elif pe < thresholds["borderline"]:
                score += 1
    else:
        print("Trailing P/E adat nem elérhető.")
    
    # Forward P/E
    fpe = info.get("forwardPE")
    if fpe is not None:
        print("Forward P/E:", fpe)
        thresholds = get_threshold("forwardPE", sector)
        if thresholds["direction"] == "lower":
            if fpe < thresholds["ideal"]:
                score += 2
            elif fpe < thresholds["borderline"]:
                score += 1
    else:
        print("Forward P/E adat nem elérhető.")
    
    # PEG Ratio
    peg = info.get("pegRatio")
    if peg is not None:
        print("PEG Ratio:", peg)
        thresholds = get_threshold("pegRatio", sector)
        if thresholds["direction"] == "lower":
            if peg < thresholds["ideal"]:
                score += 2
            elif peg < thresholds["borderline"]:
                score += 1
    else:
        print("PEG Ratio adat nem elérhető.")
    
    # Return on Equity (ROE)
    roe = info.get("returnOnEquity")
    if roe is not None:
        print("Return on Equity (ROE):", roe)
        thresholds = get_threshold("returnOnEquity", sector)
        if thresholds["direction"] == "higher":
            if roe > thresholds["ideal"]:
                score += 2
            elif roe > thresholds["borderline"]:
                score += 1
    else:
        print("ROE adat nem elérhető.")
    
    # Profit Margin
    pm = info.get("profitMargins")
    if pm is not None:
        print("Profit Margin:", pm)
        thresholds = get_threshold("profitMargins", sector)
        if thresholds["direction"] == "higher":
            if pm > thresholds["ideal"]:
                score += 2
            elif pm > thresholds["borderline"]:
                score += 1
    else:
        print("Profit Margin adat nem elérhető.")
    
    # Dividend Yield
    dy = info.get("dividendYield")
    if dy is not None:
        print("Dividend Yield:", dy)
        thresholds = get_threshold("dividendYield", sector)
        if thresholds["direction"] == "higher":
            if dy > thresholds["ideal"]:
                score += 2
            elif dy > thresholds["borderline"]:
                score += 1
    else:
        print("Dividend Yield adat nem elérhető.")
    
    # Revenue Growth (bevételek növekedése)
    rev_growth = info.get("revenueGrowth")
    if rev_growth is not None:
        print("Revenue Growth:", rev_growth)
        # Feltételezzük, hogy az ideális növekedési ütem 10% felett van
        if rev_growth > 0.10:
            score += 2
        elif rev_growth > 0.0:
            score += 1
    else:
        print("Revenue Growth adat nem elérhető.")
    
    # Debt-to-Equity (adósság/saját tőke arány)
    dte = info.get("debtToEquity")
    if dte is not None:
        try:
            dte_value = float(dte)
        except Exception as e:
            dte_value = None
        if dte_value is not None:
            print("Debt-to-Equity:", dte_value)
            # Feltételezzük: alacsonyabb érték jobb; például ideális, ha <50, borderline <100 (ez százalékban van megadva)
            if dte_value < 50:
                score += 2
            elif dte_value < 100:
                score += 1
        else:
            print("Debt-to-Equity értéke nem értelmezhető.")
    else:
        print("Debt-to-Equity adat nem elérhető.")
    
    return score

def evaluate_technical(ticker):
    """
    Egyszerű technikai elemzés:
      - Lekéri az elmúlt 1 év (kb. 250 kereskedési nap) adatait,
      - Számolja az 50 napos és 200 napos mozgóátlagokat,
      - Ha az aktuális ár meghaladja a 50 napos és/vagy a 200 napos MA-t, plusz pontot ad.
    """
    hist = ticker.history(period="1y")
    if hist.empty:
        print("Nincs elérhető történelmi adat.")
        return 0
    current_price = hist['Close'].iloc[-1]
    ma50 = hist['Close'].rolling(window=50).mean().iloc[-1]
    ma200 = hist['Close'].rolling(window=200).mean().iloc[-1]
    
    tech_score = 0
    print("\n--- Technikai elemzés ---")
    print("Aktuális ár:", current_price)
    print("50 napos mozgóátlag:", ma50)
    print("200 napos mozgóátlag:", ma200)
    
    if current_price > ma50:
        tech_score += 1
    if current_price > ma200:
        tech_score += 1
    
    return tech_score

def overall_investment_recommendation(overall_score):
    """
    Az összesített pontszám alapján ad egy befektetési ajánlást.
    A maximális pontszám a fundamentumok és a technikai elemzés eredményének összege.
    Ebben a példában:
      - 12 vagy több pont: Nagyon ajánlott befektetés.
      - 8-11 pont: Közepesen ajánlott, további kutatás javasolt.
      - Kevesebb, mint 8 pont: Nem ajánlott befektetés.
    """
    print("\nÖsszpontszám:", overall_score)
    if overall_score >= 12:
        return "Nagyon ajánlott befektetés."
    elif overall_score >= 8:
        return "Közepesen ajánlott befektetés, de további kutatás javasolt."
    else:
        return "Nem ajánlott befektetés."

def main():
    ticker_code = input("Adja meg a ticker kódot (pl. AAPL, MSFT, stb.): ").strip().upper()
    ticker = yf.Ticker(ticker_code)
    
    try:
        info = ticker.info
    except Exception as e:
        print("Hiba történt az adatok lekérése során:", e)
        return
    
    if not info:
        print("Nem sikerült lekérni az adatokat a megadott ticker kódhoz.")
        return
    
    name = info.get("shortName") or info.get("longName")
    if name:
        print("\nVállalat neve:", name)
    else:
        print("\nVállalat neve nem elérhető.")
    
    # Fundamentális elemzés
    fundamental_score = evaluate_fundamentals(info)
    # Technikai elemzés
    technical_score = evaluate_technical(ticker)
    
    # Összesített pontszám
    overall_score = fundamental_score + technical_score
    
    recommendation = overall_investment_recommendation(overall_score)
    print("\nBefektetési ajánlás:", recommendation)

if __name__ == "__main__":
    main()



Vállalat neve: Microsoft Corporation
Szektor: Technology

--- Fundamentális mutatók ---
Trailing P/E: 33.44561
Forward P/E: 27.608767
PEG Ratio adat nem elérhető.
Return on Equity (ROE): 0.35604
Profit Margin: 0.35608003
Dividend Yield: 0.0075
Revenue Growth: 0.16
Debt-to-Equity: 33.657

--- Technikai elemzés ---
Aktuális ár: 415.05999755859375
50 napos mozgóátlag: 431.30549560546876
200 napos mozgóátlag: 424.6802081298828

Összpontszám: 10

Befektetési ajánlás: Közepesen ajánlott befektetés, de további kutatás javasolt.
